<a href="https://colab.research.google.com/github/Toan02Ky-UIT/ComputerVisionProject/blob/main/Transfer_l%E1%BA%A7n_th%E1%BB%AD_%C4%91%E1%BA%A7u_ti%C3%AAn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

path = kagglehub.dataset_download("mohamedmaher5/vehicle-classification")
path

100%|██████████| 827M/827M [00:21<00:00, 39.5MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/mohamedmaher5/vehicle-classification/versions/1'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!mkdir -p "/content/drive/MyDrive/vehicle1"

In [ ]:
!cp -r "{path}" "/content/drive/MyDrive/vehicle"

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import copy
import time
import os


In [ ]:
data_dir = "/content/drive/MyDrive/vehicle1/1/Vehicles"
batch_size = 32
img_size = 224

train_tf = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

test_tf = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

full_dataset = datasets.ImageFolder(data_dir)

n = len(full_dataset)
train_len = int(n * 0.8)
val_len = int(n * 0.1)
test_len = n - train_len - val_len

train_subset, val_subset, test_subset = random_split(full_dataset, [train_len, val_len, test_len])

train_subset.dataset = datasets.ImageFolder(data_dir, transform=train_tf)
val_subset.dataset  = datasets.ImageFolder(data_dir, transform=test_tf)
test_subset.dataset = datasets.ImageFolder(data_dir, transform=test_tf)

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

num_classes = len(full_dataset.classes)

In [ ]:
def get_model(name, num_classes):
    if name == "resnet50":
        model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        in_f = model.fc.in_features
        model.fc = nn.Linear(in_f, num_classes)

    elif name.startswith("effnet"):
        eff = {
            "effnet_b0": models.efficientnet_b0,
            "effnet_b1": models.efficientnet_b1,
            "effnet_b2": models.efficientnet_b2,
            "effnet_b3": models.efficientnet_b3,
        }
        model = eff[name](weights="IMAGENET1K_V1")
        in_f = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_f, num_classes)

    for param in model.parameters():
        param.requires_grad = False
    for param in model.fc.parameters() if name == "resnet50" else model.classifier.parameters():
        param.requires_grad = True

    return model

In [ ]:
def train_model(model, name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=3)
    criterion = nn.CrossEntropyLoss()

    best_weights = copy.deepcopy(model.state_dict())
    best_acc = 0
    best_loss = float("inf")
    es_loss = float("inf")
    patience, stop_counter = 7, 0

    for epoch in range(200):
        model.train()
        train_loss, correct, total = 0, 0, 0

        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            out = model(imgs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(out, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_loss /= total

        model.eval()
        correct, total, val_loss = 0, 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                out = model(imgs)
                val_loss += criterion(out, labels).item() * imgs.size(0)
                _, preds = torch.max(out, 1)
                total += labels.size(0)
                correct += (preds == labels).sum().item()

        val_acc = correct / total
        val_loss /= total
        scheduler.step(val_loss)

        print(f"{name} → Epoch {epoch+1} | train acc={train_acc:.4f} | train loss={train_loss:.4f} | "
          f"val acc={val_acc:.4f} | val loss={val_loss:.4f} | lr={optimizer.param_groups[0]['lr']:.6f}")


        if (val_acc > best_acc) or (val_acc == best_acc and val_loss < best_loss):
            best_acc = val_acc
            best_loss = val_loss
            best_weights = copy.deepcopy(model.state_dict())
            torch.save(best_weights, f"/content/drive/MyDrive/vehicle1/{name}_best.pt")
            print("  ✓ Saved best checkpoint")

        if val_loss < es_loss:
            es_loss = val_loss
            stop_counter = 0
        else:
            stop_counter += 1
            if stop_counter >= patience:
                print("  ⛔ Early stopping")
                break

    model.load_state_dict(best_weights)
    return model


In [ ]:
def evaluate(model, name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            out = model(imgs)
            _, preds = torch.max(out, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(preds.cpu().numpy())

    print(f"\n=== {name} — Metrics ===")
    print(classification_report(y_true, y_pred, target_names=full_dataset.classes, digits=4))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

In [ ]:
model_list = ["resnet50", "effnet_b3"]

for m in model_list:
    print(f"\n================ RUN MODEL: {m} ================")
    model = get_model(m, num_classes)
    trained = train_model(model, m)
    evaluate(trained, m)



================ RUN MODEL: resnet50 ================
resnet50 → Epoch 1 | train acc=0.9309 | train loss=0.4683 | val acc=0.9857 | val loss=0.1218 | lr=0.001000
  ✓ Saved best checkpoint
resnet50 → Epoch 2 | train acc=0.9830 | train loss=0.1121 | val acc=0.9928 | val loss=0.0644 | lr=0.001000
  ✓ Saved best checkpoint
resnet50 → Epoch 3 | train acc=0.9884 | train loss=0.0699 | val acc=0.9946 | val loss=0.0547 | lr=0.001000
  ✓ Saved best checkpoint
resnet50 → Epoch 4 | train acc=0.9935 | train loss=0.0519 | val acc=0.9928 | val loss=0.0416 | lr=0.001000
resnet50 → Epoch 5 | train acc=0.9944 | train loss=0.0419 | val acc=0.9946 | val loss=0.0341 | lr=0.001000
  ✓ Saved best checkpoint
resnet50 → Epoch 6 | train acc=0.9953 | train loss=0.0336 | val acc=0.9946 | val loss=0.0364 | lr=0.001000
resnet50 → Epoch 7 | train acc=0.9949 | train loss=0.0283 | val acc=0.9928 | val loss=0.0284 | lr=0.001000
resnet50 → Epoch 8 | train acc=0.9973 | train loss=0.0251 | val acc=0.9964 | val loss=0.0275

100%|██████████| 47.2M/47.2M [00:00<00:00, 164MB/s]
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


effnet_b3 → Epoch 1 | train acc=0.8824 | train loss=0.6501 | val acc=0.9839 | val loss=0.2211 | lr=0.001000
  ✓ Saved best checkpoint
effnet_b3 → Epoch 2 | train acc=0.9477 | train loss=0.2386 | val acc=0.9857 | val loss=0.1357 | lr=0.001000
  ✓ Saved best checkpoint
effnet_b3 → Epoch 3 | train acc=0.9580 | train loss=0.1742 | val acc=0.9875 | val loss=0.1013 | lr=0.001000
  ✓ Saved best checkpoint
effnet_b3 → Epoch 4 | train acc=0.9678 | train loss=0.1367 | val acc=0.9857 | val loss=0.0852 | lr=0.001000
effnet_b3 → Epoch 5 | train acc=0.9676 | train loss=0.1250 | val acc=0.9839 | val loss=0.0742 | lr=0.001000
effnet_b3 → Epoch 6 | train acc=0.9711 | train loss=0.1097 | val acc=0.9857 | val loss=0.0705 | lr=0.001000
effnet_b3 → Epoch 7 | train acc=0.9725 | train loss=0.1049 | val acc=0.9892 | val loss=0.0636 | lr=0.001000
  ✓ Saved best checkpoint
effnet_b3 → Epoch 8 | train acc=0.9716 | train loss=0.0991 | val acc=0.9910 | val loss=0.0585 | lr=0.001000
  ✓ Saved best checkpoint
effnet